In [5]:
# Import necessary libraries
import json
import random
import re
import spacy
import pandas as pd

In [8]:
# Load the spaCy English language model
nlp = spacy.load('en_core_web_sm')

In [6]:
# Load the Excel file into a pandas DataFrame

path = 'data/validated/Accommodation/Accomodation.xlsx'

acc_df = pd.read_excel(path)
acc_df.head()

,Name,Description,https://www.booking.com/accommodation/region/in/goa.en-gb.html?aid=373420&label=goa-e%2AOApHk984VZ4sLCglKSYQS139971262944%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-433186819%3Alp1007768%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ydj4ck6nIj5YRjPru0SpaEk&sid=0dbd257ff0d64bb6e1c2e024bba6c002&keep_landing=1&gclid=CjwKCAjw6IiiBhAOEiwALNqncYnGy9c7heEgdFbyiEFCfjbH9KhxQfiWi4NTerW2sdDpOoebs0XnxBoCmX4QAvD_BwE&
0,Bella Pensao,"Offering mountain views, Bella Pensao in Parra...",NaN
1,Dreams of Palolem Beach Huts Goa,"Situated in Canacona, 200 metres from Palolem ...",NaN
2,Ronnie's Studio Apartment,"Situated 2.5 km from Baga Beach, Ronnie's Stud...",NaN
3,Zulu Land cottages - near Curlies beach shack ...,"Located in Anjuna, a few steps from Anjuna Bea...",NaN
4,Mojigao,"With garden views, Mojigao is located in Assag...",NaN


In [7]:
acc_df = acc_df.iloc[:, :2]
acc_df.head()

,Name,Description
0,Bella Pensao,"Offering mountain views, Bella Pensao in Parra..."
1,Dreams of Palolem Beach Huts Goa,"Situated in Canacona, 200 metres from Palolem ..."
2,Ronnie's Studio Apartment,"Situated 2.5 km from Baga Beach, Ronnie's Stud..."
3,Zulu Land cottages - near Curlies beach shack ...,"Located in Anjuna, a few steps from Anjuna Bea..."
4,Mojigao,"With garden views, Mojigao is located in Assag..."


In [9]:
# Define the tags and their associated keywords
tags = {
    "wifi": ["wifi", "internet"],
    "fort": ["fort", "historic"],
    "beach": ["beach", "sea"],
    "parking": ["parking", "car"],
    "gym": ["gym", "fitness"],
}

# Initialize the intents dictionary
intents = {"intents": []}

In [16]:
# Loop through each tag
for tag, keywords in tags.items():
    # Find the hotels that match the tag keywords
    matched_hotels = []
    for _, row in acc_df.iterrows():
        for keyword in keywords:
            if re.search(keyword, row["Description"], re.IGNORECASE):
                matched_hotels.append(row)
                break

    # Generate the patterns and responses for the tag
    patterns = []
    responses = []
    for hotel in matched_hotels:
        doc = nlp(hotel["Description"])
        pattern = f"where is {hotel['Name']}"
        if any(token.text == "location" for token in doc):
            pattern = f"what is the location of {hotel['Name']}"
        patterns.append(pattern)
        locations = [ent.text for ent in doc.ents if ent.label_ == "GPE"]
        if locations:
            response = f"{hotel['Name']} is located in {', '.join(locations)} in Goa."
        else:
            response = f"{hotel['Name']} is located in Goa."
        responses.append(response)

    # Add the tag to the intents dictionary
    intents["intents"].append({"tag": tag, "patterns": patterns, "responses": responses})

In [17]:
# Write the intents dictionary to a file
with open("intents.json", "w") as f:
    json.dump(intents, f, indent=2)